In [32]:
import pandas as py
import folium

# Libraries for Neo4j
from neo4j.v1 import GraphDatabase
from neo4j.v1 import exceptions

import secrets

In [ ]:
url = "http://localhost:7474/db/data/index/node/"
payload= {
  "name" : "twitter_places",
  "config" : {
    "provider" : "spatial",
    "geometry_type" : "point",
    "lat" : "lat",
    "lon" : "lon"
  }
}

r = requests.post(url, data=json.dumps(payload), headers=headers)

In [4]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=(secrets.neo4j_username, secrets.neo4j_pwd))
session = driver.session()

In [39]:
result = session.run("""
WITH point({ longitude: -93.26204, latitude: 44.97061}) AS target_point
MATCH (u:User)<-[:TWEETED_BY]-(t:Tweet)-[:LOCATED_AT]->(p:Place)
WHERE distance(p.centroid, target_point) < 20000
RETURN t.tweet_id, t.text, t.retweet_count, u.user_id, u.user_name, 
p.place_id, p.name, p.country_code, p.place_type, p.centroid.latitude AS lat, p.centroid.longitude AS long
""")
results_df = py.DataFrame(result.data())

In [40]:
results_df

,lat,long,p.country_code,p.name,p.place_id,p.place_type,t.retweet_count,t.text,t.tweet_id,u.user_id,u.user_name
0,44.963825,-93.044991,US,Maplewood,fea1c768c9a65cc7,city,0,#Hello2018,947694187247493122,1315799131,None
1,44.963825,-93.044991,US,Maplewood,fea1c768c9a65cc7,city,0,Happy New Year everybody! @ MYTH LIVE https://...,947709864524083202,2171151960,None
2,44.963825,-93.044991,US,Maplewood,fea1c768c9a65cc7,city,0,I love girls. This is probably my favorite sho...,947701982306938881,244432029,None
3,44.963825,-93.044991,US,Maplewood,fea1c768c9a65cc7,city,0,More all female lineups in 2018,947707606281478144,244432029,None
4,44.963825,-93.044991,US,Maplewood,fea1c768c9a65cc7,city,0,Hoooooly shit @brookecandy ???? https://t.co/e...,947699107493269504,244432029,None
5,44.963825,-93.044991,US,Maplewood,fea1c768c9a65cc7,city,0,Black spotting with @OneCoolBroad and @DrKrist...,947706027574808576,184626149,None
6,44.944919,-93.096109,US,The Saint Paul Hotel,07d9f31bc8882000,poi,0,What do hardcore DIY?ers do on New Years Eve.....,947696488347553792,600368834,None
7,44.941516,-93.105649,US,St Paul,60e2c37980197297,city,0,"See our latest #SaintPaul, MN #job and click t...",947694818834096129,1072440804,None
8,44.941516,-93.105649,US,St Paul,60e2c37980197297,city,0,"When we get lost in our importance, we lose si...",947702737860550656,102852308,None
9,44.941516,-93.105649,US,St Paul,60e2c37980197297,city,0,Not a beach. New Year's in MN involves frigid ...,947702711495155713,19262652,None


In [56]:
# Initialize map
m = folium.Map(
    location=[results_df["lat"].mean(), results_df["long"].mean()],
    zoom_start=8,
    tiles='Stamen Terrain'
)

# Iterate over tweets, adding markers to map for each tweet
for index, row in results_df.iterrows():
    folium.Marker(
        location=[row['lat'], row['long']],
        popup = folium.Popup(row['t.text'], parse_html=True)
    ).add_to(m)

# Display map
m